In [2]:
#Mount
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [1]:
import torch
from transformers import pipeline
import librosa
import soundfile as sf

# librosa - audio
device = 0 if torch.cuda.is_available() else "cpu"
print("device:", device)


device: 0


##Catch audio record from the brauser



with help java script:

In [ ]:
!pip install pydub
!pip install ffmpeg-python
from IPython.display import Javascript

RECORD_JS = r"""
const div = document.createElement('div');
div.innerHTML = `
  <button id="recordButton">Record</button>
  <button id="stopButton" disabled>Stop</button>
  <span id="status" style="margin-left:20px;"></span>
`;
document.body.appendChild(div);

var mediaRecorder;
var recordedBlobs;

document.getElementById('recordButton').onclick = async () => {
  document.getElementById('status').innerText = 'Recording...';
  recordedBlobs = [];
  let stream = await navigator.mediaDevices.getUserMedia({audio: true});
  mediaRecorder = new MediaRecorder(stream);
  mediaRecorder.start();

  mediaRecorder.ondataavailable = (event) => {
    if (event.data && event.data.size > 0) {
      recordedBlobs.push(event.data);
    }
  };

  document.getElementById('recordButton').disabled = true;
  document.getElementById('stopButton').disabled = false;
};

document.getElementById('stopButton').onclick = () => {
  mediaRecorder.stop();
  document.getElementById('status').innerText = 'Recording stopped.';
  document.getElementById('recordButton').disabled = false;
  document.getElementById('stopButton').disabled = true;
};
"""

display(Javascript(RECORD_JS))


from google.colab import output
from base64 import b64decode

recorded_data = None  # Глобальная переменная для хранения base64-данных

def _set_audio_data(b64_audio):
    global recorded_data
    recorded_data = b64_audio

# Регистрируем колбэк, чтобы из JS можно было вызвать Python-функцию
output.register_callback('notebook.get_audio', _set_audio_data)

def fetch_audio_from_browser():
    """
    Запускает JavaScript, который формирует base64-строку из recordedBlobs
    и вызывает Python-функцию через output.invokeFunction()
    """
    print("Запускаю JS для выгрузки аудио...")
    code = r"""
    var blob = new Blob(recordedBlobs, {type: 'audio/webm'});
    var reader = new FileReader();
    reader.onloadend = function() {
        // reader.result это dataURL (например: "data:audio/webm;base64,AAA...")
        google.colab.kernel.invokeFunction('notebook.get_audio', [reader.result], {});
    };
    reader.readAsDataURL(blob);
    """
    output.eval_js(code)
    print("Готово! Теперь переменная recorded_data в Python имеет base64-данные.")



fetch_audio_from_browser()

# Декодируем base64-строку (убираем "data:audio/webm;base64," в начале)
if recorded_data:
    header, encoded = recorded_data.split(',', 1)
    audio_bytes = b64decode(encoded)

    # Сохраняем как webm
    with open('recorded_audio.webm', 'wb') as f:
        f.write(audio_bytes)

    print("Аудио сохранено в recorded_audio.webm")
else:
    print("Похоже, нет записанных данных. Убедитесь, что вы нажали Record -> Stop, а затем вызвали fetch_audio_from_browser().")


import ffmpeg

input_file = 'recorded_audio.webm'
output_file_wav = 'recorded_audio.wav'

(
    ffmpeg
    .input(input_file)
    .output(output_file_wav, format='wav')
    .run()
)

print(f"Сконвертировано в {output_file_wav}")


###ASR:

In [ ]:
asr_pipeline = pipeline(
    task="automatic-speech-recognition",
    model="openai/whisper-small",
    device=device
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
import librosa
import soundfile as sf
input_audio_path = "/content/test.mp3"  #

audio, sr = librosa.load(input_audio_path, sr=None)

print(f" {len(audio)}")
print(f" {sr}")

duration_sec = 15
num_samples = duration_sec * sr

if len(audio) < num_samples:
    num_samples = len(audio)

audio_15s = audio[:num_samples]

print(f"Lengh {len(audio_15s)}")

output_audio_path = "/content/my_audio_15s.wav"
sf.write(output_audio_path, audio_15s, sr)

print(f"Save as {output_audio_path}")


Исходная длина аудио в сэмплах: 3082752
Частота дискретизации: 44100
Длина обрезанного аудио в сэмплах: 661500
Файл сохранён как /content/my_audio_15s.wav


In [ ]:
AUDIO_FILE = "/content/my_audio_15s.wav"

# run
result = asr_pipeline(AUDIO_FILE)

print("Text:")
print(result["text"])

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:512: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Text:
 When I was 17, I read a quote that went something like,


# LLM Bielik 2.2


In [2]:
llm_model_name = "speakleash/Bielik-11B-v2.2-Instruct"
recognized_text = "test - test - test "
llm_pipeline = pipeline(
    "text-generation",
    model=llm_model_name,
    device=device
)

prompt = f"To jest rozpoznana fraza: '{recognized_text}'. Sformułuj krótką odpowiedź."

llm_output = llm_pipeline(prompt, max_length=100, num_return_sequences=1)

# Извлекаем готовый текст:
generated_text = llm_output[0]["generated_text"]
print("Answer LLM (Bielik 2.2):")
print(generated_text)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/598 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/37.3k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/3.49k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

Device set to use cuda:0


OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 191.06 MiB is free. Process 56838 has 14.56 GiB memory in use. Of the allocated memory 14.46 GiB is allocated by PyTorch, and 1.46 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

### TTS:

In [ ]:
tts_pipeline = pipeline("text-to-speech", model="microsoft/speecht5_tts", device=device)

speech = tts_pipeline(generated_text)
with open("/content/llm_answer_speecht.wav", "wb") as f:
    f.write(speech[0]["generated_samples"])
